# 3D Basament LRD

In [3]:
DATADIR='data/' # Directory with the data
FIGURESDIR='figures/' # Figures produced

In [4]:
import functions
from functions import *

deltacontourn=pd.read_csv(DATADIR+'deltacontourn.csv')
xyzcontourn=coordinates(DATADIR+'deltacontourn.csv',[0,1,2])

contourn_bounds=bounds(xyzcontourn)

contourn_poly=Polygon(zip(xyzcontourn[0],xyzcontourn[1]))

We read the data from those boreholes that reach the basament.

In [7]:
basament=pd.read_csv(DATADIR+'basament.csv')

Now we apply the function 'coordinates' to extract the coordinates from the data.

In [8]:
xyzbasament=coordinates(DATADIR+'basament.csv',[1,2,3])

The function `nearby` reduces de coordinates to those inside the delta contour.

In [9]:
xyzbasament_near=nearby(xyzbasament,contourn_poly,500)

We are going to adapt the bounds keeping in mind the bound given by the data in the basement, so first we apply the `bounds` function to the basement and then we use a new function `bounds_join(b1,b2)` which calculates new bounds of two lists of bounds.

In [11]:
basament_bounds=bounds(xyzbasament)

In [12]:
new_bounds=bounds_join(contourn_bounds,basament_bounds)

We apply the interpolation.

In [13]:
basament_itp=interpolation(xyzbasament,100,new_bounds)

We want to represent the basement surface only within the delta contour. We have defined a 'slice' function to slice data, such as the output of the 'interpolation' function, and take only those points in a polynomial region. We apply this function to the `basament_itp` list.

In [15]:
cbasament_itp=cutting(basament_itp,contourn_poly,500)

We also want to add a mark at the position of each borehole at height 7.

In [16]:
xyzdat=coordinates(DATADIR+'boreholes.csv',[1,2,3])
xyzdat_near=nearby(xyzdat,contourn_poly,300)

The 'data_p' function is now used to obtain the data for the points in the figure.

In [19]:
data_points=data_p([xyzdat_near,xyzbasament_near],['boreholes location','basement points'],['darkblue','red'],['cross','circle'],2)

To draw the basament surface we use the plotly.graph_objects fuction 'Surface'.

In [20]:
fig=go.Figure(data=data_points)
fig.add_trace(go.Scatter3d(x=xyzcontourn[0],y=xyzcontourn[1],z=xyzcontourn[2], mode="lines",
                           line_width=5,
                           name='Delta Contour',
                           marker = dict(
                               size = 4,
                               color = 'black'
                               )
                          )
             )

fig.add_trace(go.Surface(z=cbasament_itp[0],x=cbasament_itp[1],y=cbasament_itp[2], 
                        opacity = 0.7,
                        colorscale='brwnyl',
                        name='superficie basamento',
                        #label='superficie basamento',
                        showscale=False
                        )             
             )

fig.update_layout( title="Pliocene basement Llobregat Delta, Z scale is x 50",
    scene=dict(aspectratio=dict(x=2, y=2, z=0.5),
               xaxis = dict(range=[contourn_bounds[0]-2000,contourn_bounds[1]+2000],),
               yaxis = dict(range=[contourn_bounds[2]-2000,contourn_bounds[3]+2000])
                            )
                 )

#fig.show()
go_offline.plot(fig,filename=FIGURESDIR+'3D_Basament_LRD.html',validate=True, auto_open=False)

'figures/3D_Basament_LRD.html'